In [2]:
import torch
import numpy as np
import random

from main.models import ConvNet, get_model_constructor
from main.utils import save_experiment
from dataclasses import dataclass
from main.active_learning import run_active_learning
from main.prepare_data import create_dataloaders
from configurations import get_config

%reload_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# pick 10 super random numbers
fixed_seeds = [12345, 23456, 34567, 45678, 56789, 67890, 78901, 89012, 90123, 12340]

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [4]:
config, experiment_name = get_config(
    method='similarity_kmeans',
    min_samples=20,
    max_samples=200,
    acquisition_batch_size=10,
    dataset='mnist',
    )

config.extract_pool = 59900

experiment_name = experiment_name

print(f'Running {experiment_name} experiment')
save_results = True
num_runs = 5

Running similarity_kmeans_mnist_20_to_200_B=10 experiment


In [5]:
for i in range(num_runs):
    current_seed = fixed_seeds[i]
    torch.manual_seed(current_seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(current_seed)
        torch.cuda.manual_seed_all(current_seed)

    # load data
    train_loader, test_loader, pool_loader, active_learning_data = create_dataloaders(config)

    print(len(train_loader.dataset), len(test_loader.dataset), len(pool_loader.dataset))
    
    # get results
    results = run_active_learning(
        train_loader=train_loader,
        test_loader=test_loader, 
        pool_loader=pool_loader,
        active_learning_data=active_learning_data,
        model_constructor=get_model_constructor(config.al_method, config.dataset), 
        config=config, 
        device=device
        )

    # save results and configuration
    if save_results:
        experiment_id = experiment_name + '_' + str(i + 1)
        save_experiment(config, results, experiment_id, base_dir='pseudo_label_experiments')
    
    del train_loader, test_loader, pool_loader, active_learning_data

c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\torch\utils\data\sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."


20 10000 80


Training Set Size:  10%|█         | 20/200 [00:00<?, ?it/s]2024-08-31 12:02:28,402 - INFO - Training set size: 20, Test set accuracy: 61.62, Test set loss: 0.0035
c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
c:\Users\vince\Documents\Statistics\TT\msc_thesis\.venv\Lib\site-packages\laplace\baselaplace.py:409: UserWarning: By default `link_approx` is `probit`. Make sure to set it equals to the way you want to call `la(test_data, pred_type=..., link_approx=...)`.
  warnings.warn(
Training Set Size:  15%|█▌        | 30/200 [00:17<04:54,  1.73s/it]

HP DEVICE: cpu
Precision DEVICE: cpu
torch.float32
#Samps	Total Distance
1	50.857357025146484
2	12.149543762207031
3	11.793352127075195
4	11.175128936767578
5	10.823549270629883
6	10.351934432983398
7	9.975040435791016
8	9.520711898803711
9	9.201507568359375
Dataset indices:  [51303 48691 39082 13779 48193 35465 27389 20952  2517  4084]
Scores:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels:  tensor([0, 7, 7, 0, 3, 0, 3, 3, 2, 8])


KeyboardInterrupt: 